In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import numpy as np
from numpy import random
database = 'database.sqlite'
conn = sqlite3.connect('../database/database.sqlite')

In [8]:
def tabela_mando_home(stageI, stageF, league_id, season, team_id):
    sql = """select team, count(case when gf > gs then 1 end) V,
    count(case when gs > gf then 1 end) D,
    count(case when gf = gs then 1 end) E,
    sum(gf) GF, avg(gf) as AVG_GF ,  sum(gs) as GS  , avg(gs) as AVG_GS,
    count(case when gs>0 then 1 end) as J1GS, 
    count (case when gf>0 then 1 end) as J1GF,
    count (case when (gf>0 and gs>0) then 1 end) as BTTS,
    count (case when (gf + gs>2) then 1 end) as Over2,
    count (case when (gf + gs>3) then 1 end) as Over3,
    count (case when (gf + gs>4) then 1 end) as Over4,
    count (case when (gf>gs and gs=0) then 1 end) as WinningCleanSheet,
    count (case when (gs>gf and gf=0) then 1 end) as LosingFS,
    count (case when (gf=0) then 1 end) as FailedScore,
    count (case when (gs=0) then 1 end) as CleanSheet,
    avg (gf+gs) as AVG_GOALS from 
    (select home_team_api_id team, home_team_goal gf, away_team_goal gs from Match 
    where (stage between """+str(stageI)+""" and """+str(stageF)+""" ) and (season='"""+str(season)+"""') and league_id ="""+str(league_id)+"""  and home_team_api_id="""+str(team_id)+""") a 
    group by team order by V desc , GF-GS desc, GF desc;"""

    return pd.read_sql(sql, conn)

def tabela_mando_away(stageI, stageF, league_id, season, team_id):
    sql = """select team as team, count(case when gf > gs then 1 end) V,
    count(case when gs > gf then 1 end) D,
    count(case when gf = gs then 1 end) E,
    sum(gf) GF, avg(gf) as AVG_GF ,  sum(gs) as GS  , avg(gs) as AVG_GS,
    count(case when gs>0 then 1 end) as J1GS, 
    count (case when gf>0 then 1 end) as J1GF,
    count (case when (gf>0 and gs>0) then 1 end) as BTTS,
    count (case when (gf + gs>2) then 1 end) as Over2,
    count (case when (gf + gs>3) then 1 end) as Over3,
    count (case when (gf + gs>4) then 1 end) as Over4,
    count (case when (gf>gs and gs=0) then 1 end) as WinningCleanSheet,
    count (case when (gs>gf and gf=0) then 1 end) as LosingFS,
    count (case when (gf=0) then 1 end) as FailedScore,
    count (case when (gs=0) then 1 end) as CleanSheet,
    avg (gf+gs) as AVG_GOALS from 
    (select away_team_api_id team, away_team_goal gf, home_team_goal gs from Match 
    where (stage between """+str(stageI)+""" and """+str(stageF)+""" ) and (season='"""+str(season)+"""') and league_id ="""+str(league_id)+"""  and away_team_api_id="""+str(team_id)+""") a 
    group by team order by V desc , GF-GS desc, GF desc;"""
    return pd.read_sql(sql, conn)


def tabela(stageI, stageF, league_id, season, team_id):
    sql= """select team , count(case when gf > gs then 1 end) V,
    count(case when gs > gf then 1 end) D,
    count(case when gf = gs then 1 end) E,
    sum(gf) GF, avg(gf) as AVG_GF ,  sum(gs) as GS  , avg(gs) as AVG_GS,
    count(case when gs>0 then 1 end) as J1GS, 
    count (case when gf>0 then 1 end) as J1GF,
    count (case when (gf>0 and gs>0) then 1 end) as BTTS,
    count (case when (gf + gs>2) then 1 end) as Over2  ,
    count (case when (gf + gs>3) then 1 end) as Over3  ,
    count (case when (gf + gs>4) then 1 end) as Over4  ,
    count (case when (gf>gs and gs=0) then 1 end) as WinningCleanSheet,
    count (case when (gs>gf and gf=0) then 1 end) as LosingFS,
    count (case when (gf=0) then 1 end) as FailedScore,
    count (case when (gs=0) then 1 end) as CleanSheet,
    avg (gf+gs) as AVG_GOALS from 
    (select away_team_api_id as team, home_team_goal as  gs , away_team_goal as gf  from Match 
    where (stage between """+str(stageI)+""" and """+str(stageF)+""" ) and (season='"""+str(season)+"""') and league_id ="""+str(league_id)+""" and away_team_api_id="""+str(team_id)+"""
    union all select home_team_api_id as team,  away_team_goal as gs, home_team_goal as gf from Match 
    where (stage between """+str(stageI)+""" and """+str(stageF)+""" ) and (season='"""+str(season)+"""') and league_id ="""+str(league_id)+""" and home_team_api_id="""+str(team_id)+""") a group by team order by V desc , GF-GS desc,GF desc;"""
    return pd.read_sql(sql, conn)
def rank_pos(team_id,data):
    i=1
    ult=0
    k=0    
    for index, row in data.iterrows():
            if ult == row['tg']:
                k=k+1
                if row['team']==team_id:
                    return i-1
            else:
                if row['team']==team_id:
                    return i+k
                ult = row['tg'] 
                i=i+k+1
                k=0

def rank_def_away(stageI,stageF,league_id,season,team_id):
    
    sql= """select away_team_api_id as team, sum (home_team_goal) as tg  from Match 
    where (stage between """+str(stageI)+""" and """+str(stageF)+""") and (season='"""+str(season)+"""') and league_id="""+str(league_id)+"""   
    group by team order by tg desc;"""
    a = pd.read_sql(sql,conn)
    return a
    #return rank_pos(str(team_id),a)

def rank_def_home(stageI,stageF,league_id,season,team_id):
    
    sql= """select home_team_api_id as team, sum (away_team_goal) as tg  from Match 
    where (stage between """+str(stageI)+""" and """+str(stageF)+""") and (season='"""+str(season)+"""') and league_id="""+str(league_id)+"""   
    group by team order by tg desc;"""
    a = pd.read_sql(sql,conn)
    return a
    #return rank_pos(str(team_id),a)

def rank_atk_away(stageI,stageF,league_id,season,team_id):
    
    sql= """select away_team_api_id as team, sum (away_team_goal) as tg  from Match 
    where (stage between """+str(stageI)+""" and """+str(stageF)+""") and (season='"""+str(season)+"""') and league_id="""+str(league_id)+"""   
    group by team order by tg desc;"""
    a = pd.read_sql(sql,conn)
    return a
    #return rank_pos(str(team_id),a)

def rank_atk_home(stageI,stageF,league_id,season,team_id):
    
    sql= """select home_team_api_id as team, sum (home_team_goal) as tg  from Match 
    where (stage between """+str(stageI)+""" and """+str(stageF)+""") and (season='"""+str(season)+"""') and league_id="""+str(league_id)+"""   
    group by team order by tg desc;"""
    a = pd.read_sql(sql,conn)
    
    return a

def last_match_date(date,team_id,season):
    sql = """select date from Match where 
    season='"""+str(season)+"""' and date<'"""+str(date)+"""' and 
    (home_team_api_id="""+str(team_id)+""" or away_team_api_id="""+str(team_id)+""") order by date desc;"""
    #print sql
    return  pd.read_sql(sql, conn).head(1)

In [3]:
sql = """select * from Match where league_id !=24558 and league_id != 19694"""                  

matches = pd.read_sql(sql,conn)
drop = matches.columns.values[-27:-1]
#Removing other betting houses odds
matches = matches.drop(drop,1)
matches= matches.drop('BSA',1)
#Raw features
matches= matches.drop(['match_api_id','id','country_id','goal', 'shoton', 'shotoff', 'foulcommit', 'card',
       'cross', 'corner', 'possession','home_player_X1', 'home_player_X2', 'home_player_X3',
        'home_player_X4', 'home_player_X5', 'home_player_X6',
        'home_player_X7', 'home_player_X8', 'home_player_X9',
        'home_player_X10', 'home_player_X11', 'away_player_X1',
        'away_player_X2', 'away_player_X3', 'away_player_X4',
        'away_player_X5', 'away_player_X6', 'away_player_X7',
        'away_player_X8', 'away_player_X9', 'away_player_X10',
        'away_player_X11', 'home_player_Y1', 'home_player_Y2',
        'home_player_Y3', 'home_player_Y4', 'home_player_Y5',
        'home_player_Y6', 'home_player_Y7', 'home_player_Y8',
        'home_player_Y9', 'home_player_Y10', 'home_player_Y11',
        'away_player_Y1', 'away_player_Y2', 'away_player_Y3',
        'away_player_Y4', 'away_player_Y5', 'away_player_Y6',
        'away_player_Y7', 'away_player_Y8', 'away_player_Y9',
        'away_player_Y10', 'away_player_Y11',
        'home_player_1','home_player_2','home_player_3','home_player_4',
        'home_player_5','home_player_6','home_player_7','home_player_8',
        'home_player_9','home_player_10','home_player_11',
        'away_player_1','away_player_2','away_player_3','away_player_4',
        'away_player_5','away_player_6','away_player_7','away_player_8',
        'away_player_9','away_player_10','away_player_11'],1)
matches = matches[matches['league_id']!= 1]

from datetime import datetime

data_final = pd.DataFrame()
delta_stage=5
matches['date']=pd.to_datetime(matches['date'], format='%Y-%m-%d %H:%M:%S.%f')

matches.sort_values(["league_id","season","stage"])

matches


,league_id,season,stage,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A
1728,1729,2008/2009,1,2008-08-17,10260,10261,1,1,1.29,5.50,11.00
1729,1729,2008/2009,1,2008-08-16,9825,8659,1,0,1.20,6.50,15.00
1730,1729,2008/2009,1,2008-08-16,8472,8650,0,1,5.50,3.60,1.67
1731,1729,2008/2009,1,2008-08-16,8654,8528,2,1,1.91,3.40,4.20
1732,1729,2008/2009,1,2008-08-17,10252,8456,4,2,1.91,3.40,4.33
1733,1729,2008/2009,1,2008-08-16,8668,8655,2,3,2.00,3.30,4.00
1734,1729,2008/2009,1,2008-08-16,8549,8586,2,1,3.20,3.40,2.25
1735,1729,2008/2009,1,2008-08-16,8559,10194,3,1,1.83,3.50,4.50
1736,1729,2008/2009,1,2008-08-16,8667,9879,2,1,2.60,3.20,2.80
1737,1729,2008/2009,1,2008-08-17,8455,8462,4,0,1.33,5.00,10.00


In [10]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print'\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix),'\r'
    # Print New Line on Complete
    if iteration == total: 
        print()

        

matches = matches[matches['league_id']!= 1]

from datetime import datetime

data_final = pd.DataFrame()
delta_stage=5
matches['date']=pd.to_datetime(matches['date'], format='%Y-%m-%d %H:%M:%S.%f')

matches.sort_values(["league_id","season","stage"])
#Loop extract data
l = len(matches)
c=0
  
printProgressBar(c, l, prefix = 'Progress:', suffix = 'Complete', length = 50)

for index, row in matches.iterrows():    
    c=c+1
    if row['stage']>delta_stage:
        local_c_mandante = tabela_mando_home(row['stage']-delta_stage,row['stage']-1,row['league_id'],row['season'],row['home_team_api_id'])
        local_c_mandante.columns = ["hlc_"+ str(col) for col in local_c_mandante.columns]
        local_mandante = tabela(row['stage']-delta_stage,row['stage']-1,row['league_id'],row['season'],row['home_team_api_id'])
        local_mandante.columns = ["hl_"+ str(col) for col in local_mandante.columns]
        global_c_mandante = tabela_mando_home(1,row['stage']-1,row['league_id'],row['season'],row['home_team_api_id'])
        global_c_mandante.columns = ["hgc_"+ str(col) for col in global_c_mandante.columns]        
        global_mandante = tabela(1,row['stage']-1,row['league_id'],row['season'],row['home_team_api_id'])
        global_mandante.columns = ["hg_"+ str(col) for col in global_mandante.columns]
        local = pd.concat([local_mandante,local_c_mandante],axis=1)
        glob =  pd.concat([global_mandante,global_c_mandante],axis=1)
        mandante =  pd.concat([glob,local],axis=1)
        mandante = mandante.drop(['hl_team','hg_team','hgc_team','hlc_team'],axis=1)
        ## Visitante ##
        local_visitante = tabela(row['stage']-delta_stage,row['stage']-1,row['league_id'],row['season'],row['away_team_api_id'])
        local_visitante.columns = ["al_"+ str(col) for col in local_visitante.columns]
        local_c_visitante = tabela_mando_away(row['stage']-delta_stage,row['stage']-1,row['league_id'],row['season'],row['away_team_api_id'])
        local_c_visitante.columns = ["alc_"+ str(col) for col in local_c_visitante.columns]
        global_visitante = tabela(1,row['stage']-1,row['league_id'],row['season'],row['away_team_api_id'])
        global_visitante.columns = ["ag_"+ str(col) for col in global_visitante.columns]
        global_c_visitante = tabela_mando_away(1,row['stage']-1,row['league_id'],row['season'],row['away_team_api_id'])
        global_c_visitante.columns = ["agc_"+ str(col) for col in global_c_visitante.columns]
        
        local = pd.concat([local_visitante,local_c_visitante],axis=1)
        glob =  pd.concat([global_visitante,global_c_visitante],axis=1)
        visitante =  pd.concat([glob,local],axis=1)
        visitante = visitante.drop(['al_team','ag_team','agc_team','alc_team'],axis=1)
        res = pd.concat([mandante,visitante],axis=1)
        res['ga_rank_def'] =  rank_pos(row['away_team_api_id'],rank_def_away(1,row['stage']-1,row['league_id'],row['season'],row['away_team_api_id']))
        res['ga_rank_atk'] =  rank_pos(row['away_team_api_id'],rank_atk_away(1,row['stage']-1,row['league_id'],row['season'],row['away_team_api_id']))
        res['gh_rank_def'] =  rank_pos(row['home_team_api_id'],rank_def_away(1,row['stage']-1,row['league_id'],row['season'],row['home_team_api_id']))
        res['gh_rank_atk'] =  rank_pos(row['home_team_api_id'],rank_atk_away(1,row['stage']-1,row['league_id'],row['season'],row['home_team_api_id']))
        res['la_rank_def'] =  rank_pos(row['away_team_api_id'],rank_def_away(row['stage']-delta_stage,row['stage']-1,row['league_id'],row['season'],row['away_team_api_id']))
        res['la_rank_atk'] =  rank_pos(row['away_team_api_id'],rank_atk_away(row['stage']-delta_stage,row['stage']-1,row['league_id'],row['season'],row['away_team_api_id']))
        res['lh_rank_def'] =  rank_pos(row['home_team_api_id'],rank_def_away(row['stage']-delta_stage,row['stage']-1,row['league_id'],row['season'],row['home_team_api_id']))
        res['lh_rank_atk'] =  rank_pos(row['home_team_api_id'],rank_atk_away(row['stage']-delta_stage,row['stage']-1,row['league_id'],row['season'],row['home_team_api_id']))
                                       
        
        
        #Calculo de dias do ultimo jogo
        if row['stage']==1 :
            res['h_date']=0
            res['a_date']=0
        else:
            h_date= last_match_date(row['date'],row['home_team_api_id'],row['season'])
            h_date['date']=pd.to_datetime(h_date['date'], format='%Y-%m-%d %H:%M:%S.%f')
            a_date= last_match_date(row['date'],row['away_team_api_id'],row['season'])
            a_date['date']=pd.to_datetime(a_date['date'], format='%Y-%m-%d %H:%M:%S.%f')
            #print row['stage']
            #print h_date['date']
            if a_date['date'].empty:
                res['a_date']=3
            else:
                res['a_date']=row['date']-a_date['date']
                
            if h_date['date'].empty:
                res['h_date']=3
            else:
                res['h_date']=row['date']-h_date['date']
                
        #Junta os dados do jogo atual com os dados gerados pelas funções anteriores.
        for field in row.index:
            res[field] = row[field]
        data_final = data_final.append(res)
        if c%150 == 0:
            printProgressBar(c, l, prefix = 'Progress:', suffix = 'Complete', length = 50)

        

    
print "FIM"
                                       
data_final.to_csv('dados2.csv')

Progress: |--------------------------------------------------| 0.0% Complete 
Progress: |--------------------------------------------------| 0.7% Complete 
Progress: |--------------------------------------------------| 1.4% Complete 
Progress: |█-------------------------------------------------| 2.1% Complete 
Progress: |█-------------------------------------------------| 2.9% Complete 
Progress: |█-------------------------------------------------| 3.6% Complete 
Progress: |██------------------------------------------------| 4.3% Complete 
Progress: |██------------------------------------------------| 5.0% Complete 
Progress: |██------------------------------------------------| 5.7% Complete 
Progress: |███-----------------------------------------------| 6.4% Complete 
Progress: |███-----------------------------------------------| 7.1% Complete 
Progress: |███-----------------------------------------------| 7.9% Complete 
Progress: |████----------------------------------------------| 8

In [11]:
data_final.columns.values

array(['hg_V', 'hg_D', 'hg_E', 'hg_GF', 'hg_AVG_GF', 'hg_GS', 'hg_AVG_GS',
       'hg_J1GS', 'hg_J1GF', 'hg_BTTS', 'hg_Over2', 'hg_Over3', 'hg_Over4',
       'hg_WinningCleanSheet', 'hg_LosingFS', 'hg_FailedScore',
       'hg_CleanSheet', 'hg_AVG_GOALS', 'hgc_V', 'hgc_D', 'hgc_E',
       'hgc_GF', 'hgc_AVG_GF', 'hgc_GS', 'hgc_AVG_GS', 'hgc_J1GS',
       'hgc_J1GF', 'hgc_BTTS', 'hgc_Over2', 'hgc_Over3', 'hgc_Over4',
       'hgc_WinningCleanSheet', 'hgc_LosingFS', 'hgc_FailedScore',
       'hgc_CleanSheet', 'hgc_AVG_GOALS', 'hl_V', 'hl_D', 'hl_E', 'hl_GF',
       'hl_AVG_GF', 'hl_GS', 'hl_AVG_GS', 'hl_J1GS', 'hl_J1GF', 'hl_BTTS',
       'hl_Over2', 'hl_Over3', 'hl_Over4', 'hl_WinningCleanSheet',
       'hl_LosingFS', 'hl_FailedScore', 'hl_CleanSheet', 'hl_AVG_GOALS',
       'hlc_V', 'hlc_D', 'hlc_E', 'hlc_GF', 'hlc_AVG_GF', 'hlc_GS',
       'hlc_AVG_GS', 'hlc_J1GS', 'hlc_J1GF', 'hlc_BTTS', 'hlc_Over2',
       'hlc_Over3', 'hlc_Over4', 'hlc_WinningCleanSheet', 'hlc_LosingFS',
       'hlc_